In [5]:
import numpy as np
import nearest_neighbors


def kfold(n, n_folds=3):
    all_idx = range(n)
    folds = np.array_split(all_idx, n_folds)
    res = []
    for k in range(n_folds):
        if k == 0:
            edu = np.hstack(folds[k+1:])
        elif k == n_folds - 1:
            edu = np.hstack(folds[:k])
        else:
            edu = np.hstack((np.hstack(folds[:k]), np.hstack(folds[k+1:])))
        val = folds[k]
        res.append((edu, val))
#         print(type(edu), type(val), type(res))
    return res
   
def knn_cross_val_score(X, y, k_list, score, cv, **kwargs):
    if cv is None:
        cv = kfold(X.shape[0], n_folds=3)
    dic = {}
    for k in k_list:
        acc = []
#         if ('strategy' in kwargs) and ('metric' in kwargs) and ('weights' in kwargs) and ('test_block_size' in kwargs):
        cls = nearest_neighbors.KNNClassifier(k=k, strategy=kwargs['strategy'], 
                                                  metric=kwargs['metric'], weights=kwargs['weights'])
#         else:
#             raise TypeError
        for fold in cv:
            X_train = np.array([X[i] for i in fold[0]])
            y_train = np.array([y[i] for i in fold[0]])
            X_test = np.array([X[i] for i in fold[1]])
            y_test = np.array([y[i] for i in fold[1]])
            
            cls.fit(X_train, y_train)
            y_predict = cls.predict(X_test)
            
            if score == 'accuracy':
                accuracy = np.sum(y_predict == y_test) / X_test.shape[0]
                acc.append(accuracy)
                
        dic[k] = np.array(acc)
    return dic


In [7]:
%timeit knn_cross_val_score(X = np.array([[1, 7, 3, 8], [9, 5, 0, 1], [2, 5, 1, 0], [3, 5, 2, 9], [2, 1, 1, 2], [1, 7, 3, 8], [9, 9, 0, 7]]), cv=None, y = np.array([1, 2, 3, 1, 2, 3, 1]), k_list=[1, 2, 3, 4], score='accuracy', strategy='my_own', weights=False, test_block_size=1000, metric='euclidean')

6.08 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
cv = kfold(7)
cv

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


[(array([3, 4, 5, 6]), array([0, 1, 2])),
 (array([0, 1, 2, 5, 6]), array([3, 4])),
 (array([0, 1, 2, 3, 4]), array([5, 6]))]